[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/normalization)

<img src='__docs/__all/notebook_rules.png' />

# Select Your IPTS 

In [1]:
from __code.normalization import *

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_roi_selection.ui')
from __code.roi_selection_ui import Interface

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_roi_selection.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_roi_selection.py


** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-13639_resonance_shawn', 'IPTS-14832', 'IPTS-15177', 'IPTS-16212', 'IPTS-16259', 'IPTS-16475', 'IPTS-16893', 'IPTS-17099', 'IPTS-17685-Siddharth', 'IPTS-17981', 'IPTS-18580', 'IPTS-18806', 'IPTS-18813', 'IPTS-19037', 'IPTS-19213', 'IPTS-19558', 'IPTS-19621-CLOCK', 'IPTS-19626', 'IPTS-19799', 'IPTS-19921-Charles', 'IPTS-19943-kashif', 'IPTS-20139-Hao-Liu', 'IPTS-20209-TOPAZ', 'IPTS-21115', 'IPTS-Das-Saikat', 'IPTS-raw', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging', 'shared', 'testing_Ed_algo_water_intake'), value='BraggEdge'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Python Import

In [7]:
%gui qt

# Select Gamma Filtering Coefficient 

In [8]:
o_gamma = GammaCoefficient()
o_gamma.select_gamma_coefficient()

HBox(children=(Label(value='Gamma Coefficient:', layout=Layout(width='20%')), FloatSlider(value=0.3, layout=Layout(width='50%'), max=1.0)))

If you need help figuring out which **gamma coefficient** to use, check the [gamma filtering tool notbeook](gamma_filtering_tool.ipynb)

# Select Images (Sample, OB, and DF)

In [9]:
files = Files()
sample_panel = SampleSelectionPanel(working_dir=system.System.get_working_dir(), gamma_coefficient=o_gamma.get_coefficient())
sample_panel.init_ui(files=files)
wizard = WizardPanel(sample_panel=sample_panel)

Label(value='Selection of All Input Files', layout=Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px'))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Sample files (mandatory) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='warning', description='Next Step>>', disabled=True, layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-21115', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), SelectMultiple(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                               |     Wed Aug  1 11:33:48 2018', ' Huggies_2cm_thick                       |     Directory', ' Huggies_2cm_thick_3rd_drop              |     Directory', ' Huggies_2nd_2cm_thick                   |     Directory', ' Huggies_3cm_5th_drop                    |     Directory', ' Huggies_3cm_thick                       |     Directory', ' Huggies_3rd_whole_tilted                |     Directory', ' Huggies_Merries                         |     Directory', ' Huggies_redo                            |     Directory', ' July25_2018                             |     Directory', ' Merries_1st_2cm_thick                   |     Directory', ' Merries_2nd                             |     Directory', ' Merries_3rd_whole                       |     Directory', ' Merries_3rd_whole_diaper_background     |     Directory', ' Merries_4th_whole_tilt                  |     Directory', ' Merries_redo                            |     Directory', ' OB1                                     |     Directory', ' OB2                                     |     Directory', ' OB3                                     |     Directory', ' Videos                                  |     Directory'), value=())), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Open Beam files (mandatory) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

...

VBox(children=(Label(value='', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-21115', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), SelectMultiple(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                               |     Wed Aug  1 11:33:48 2018', ' Huggies_2cm_thick                       |     Directory', ' Huggies_2cm_thick_3rd_drop              |     Directory', ' Huggies_2nd_2cm_thick                   |     Directory', ' Huggies_3cm_5th_drop                    |     Directory', ' Huggies_3cm_thick                       |     Directory', ' Huggies_3rd_whole_tilted                |     Directory', ' Huggies_Merries                         |     Directory', ' Huggies_redo                            |     Directory', ' July25_2018                             |     Directory', ' Merries_1st_2cm_thick                   |     Directory', ' Merries_2nd                             |     Directory', ' Merries_3rd_whole                       |     Directory', ' Merries_3rd_whole_diaper_background     |     Directory', ' Merries_4th_whole_tilt                  |     Directory', ' Merries_redo                            |     Directory', ' OB1                                     |     Directory', ' OB2                                     |     Directory', ' OB3                                     |     Directory', ' Videos                                  |     Directory'), value=())), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Dark Field files (optional) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Button(button_style='success', description='<< Previous Step', layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to previous step'), Label(value='         ', layout=Layout(width='70%')), Button(button_style='success', description='Next Step>>', layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-21115', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), SelectMultiple(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                               |     Wed Aug  1 11:33:48 2018', ' Huggies_2cm_thick                       |     Directory', ' Huggies_2cm_thick_3rd_drop              |     Directory', ' Huggies_2nd_2cm_thick                   |     Directory', ' Huggies_3cm_5th_drop                    |     Directory', ' Huggies_3cm_thick                       |     Directory', ' Huggies_3rd_whole_tilted                |     Directory', ' Huggies_Merries                         |     Directory', ' Huggies_redo                            |     Directory', ' July25_2018                             |     Directory', ' Merries_1st_2cm_thick                   |     Directory', ' Merries_2nd                             |     Directory', ' Merries_3rd_whole                       |     Directory', ' Merries_3rd_whole_diaper_background     |     Directory', ' Merries_4th_whole_tilt                  |     Directory', ' Merries_redo                            |     Directory', ' OB1                                     |     Directory', ' OB2                                     |     Directory', ' OB3                                     |     Directory', ' Videos                                  |     Directory'), value=())), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

TypeError: load() got an unexpected keyword argument 'gamma_filter'

# Select Background Region 

In [ ]:
o_norm = sample_panel.o_norm_handler
o_gui = Interface(o_norm=o_norm.o_norm)
o_gui.show()

# Normalization

In [ ]:
o_norm.run_normalization(dict_roi=o_gui.roi_selected)

# Export 

In [ ]:
o_norm.select_export_folder(ipts_folder=sample_panel.ipts_dir)

In [ ]:
o_norm.export()